In [ ]:
! pip install beautifulsoup4

In [141]:
import requests
from bs4 import BeautifulSoup, NavigableString
import bs4
import re

In [7]:
response = requests.get("https://www.pimlicoplumbers.com/")
response.status_code

200

In [ ]:
soup = BeautifulSoup(response.content, 'lxml')
site_header = soup.find('header')
bar = site_header.find('div', {"class":"navbar-wrapper mobile-header"})
menu = bar.find('ul', {'id':'main-menu'})

In [50]:
base_url = 'https://www.pimlicoplumbers.com/services/'
a = menu.find_all('a',{'class':'nav-link'})
services = []
for i in a:
    if i.text != 'Air Con ':
        services.append(i.text)

servs = []
for string in services:
    if not string in ['Home','Handyman']:
        strng = string[:-1]
        servs.append(strng.lower())
    elif string == 'Home':
        continue
    else:
        servs.append(string)

print(servs)

['plumbing', 'heating', 'electrics', 'drains', 'carpentry', 'roofing', 'building', 'bathrooms', 'Handyman', 'commercial']


In [52]:
service_info = []
for i in servs:
    urls = []
    req_url = base_url+i
    all_urls = menu.find_all('a', href = True)
    for url in all_urls:
        if req_url in url['href']:
            urls.append(url['href'])
    service_info.append({'ServiceTitle': f'{i}', 'allUrls':f'{urls}'})
print(service_info)


[{'ServiceTitle': 'plumbing', 'allUrls': "['https://www.pimlicoplumbers.com/services/plumbing/', 'https://www.pimlicoplumbers.com/services/plumbing/emergency/', 'https://www.pimlicoplumbers.com/services/plumbing/installation/', 'https://www.pimlicoplumbers.com/services/plumbing/repairs/', 'https://www.pimlicoplumbers.com/services/plumbing/damp-and-leak-detection/', 'https://www.pimlicoplumbers.com/services/plumbing/drain-blockages/', 'https://www.pimlicoplumbers.com/services/plumbing/pre-purchase-survey/']"}, {'ServiceTitle': 'heating', 'allUrls': "['https://www.pimlicoplumbers.com/services/heating/', 'https://www.pimlicoplumbers.com/services/heating/emergency/', 'https://www.pimlicoplumbers.com/services/heating/boilers/', 'https://www.pimlicoplumbers.com/services/heating/installation/', 'https://www.pimlicoplumbers.com/services/heating/maintenance/', 'https://www.pimlicoplumbers.com/services/heating/repairs/', 'https://www.pimlicoplumbers.com/services/heating/survey/']"}, {'ServiceTit

In [109]:
response = requests.get('https://www.pimlicoplumbers.com/services/plumbing/emergency/')
response.status_code

200

In [110]:
sections_to_find = ['bf-acf-module banner banner-type-split','bf-acf-module image-and-content','bf-acf-module services-grid','bf-acf-module customer-testimonials','bf-acf-module text-and-list','bf-acf-module locations-plain-list']
soup = BeautifulSoup(response.content, 'lxml')
main_content = soup.find('div', {'id':'content'})
sections = []
for section_class in sections_to_find:
  found_sections = main_content.find_all('section', class_=lambda c: section_class in c)
  sections.extend(found_sections)

for section in sections:
  section_name = " ".join(section['class'])
  print(section_name)

bf-acf-module banner banner-type-split
bf-acf-module image-and-content spacing-spaced image-left
bf-acf-module image-and-content spacing-spaced image-right
bf-acf-module customer-testimonials
bf-acf-module text-and-list
bf-acf-module locations-plain-list


In [111]:
section_to_search = sections[0]
class_name = " ".join(section_to_search['class'])
print(f"section class name : {class_name}")
if " ".join(section_to_search['class']) ==  'bf-acf-module banner banner-type-split':
    title = section_to_search.find('h1')
    title = title.text
    content = section_to_search.find('p')
    content = content.text
    image = section_to_search.find('div', class_=lambda c: 'background-desktop' in c, style=True)
    url_regex = r"url\('(.*?)'\)"
    style_attr = image['style']
    match = re.search(url_regex, style_attr)
    img_url = match.group(1)
    button = section_to_search.find('button')
    button_exists = section_to_search.find('button') is not None
    if button_exists:
        button = section_to_search.find('button')
        button_txt = button.text.strip()
    else:
        button_txt = None
print({'Title':f'{title}', 'content':f'{content}', 'CallToAction':f'{button_txt}', 'ImageUrl':f'{img_url}'})

section class name : bf-acf-module banner banner-type-split
{'Title': 'Emergency Plumbers providing 24/7 services across London', 'content': 'Do you have a plumbing emergency? Our team provide emergency plumbing services across London 24 hours a day, 365 days a year. No matter the time, our plumbers can be at your property within an hour, to professionally diagnose and resolve the problem.\n', 'CallToAction': '\nRequest Booking\n', 'ImageUrl': 'https://www.pimlicoplumbers.com/wp-content/uploads/2022/08/Emergency-plumbing-desktop-banner.jpg'}


In [118]:
section_to_search = sections[2]
class_name = " ".join(section_to_search['class'])
print(f"section class name : {class_name}")
if 'bf-acf-module image-and-content' in " ".join(section_to_search['class']):
    title = section_to_search.find('h2')
    title = title.text
    contents = section_to_search.find('div',{'class':'content'})
    text_content = []
    for child in contents.children:
        if child.text:
            text_content.append(child.text.strip())
    text = ' '.join(text_content)
    img = section_to_search.find('img')
    img_url = img['src']
    button_exists = section_to_search.find('button') is not None
    if button_exists:
        button = section_to_search.find('button')
        button_txt = button.text.strip()
    else:
        button_txt = None

print({'Title':f'{title}', 'content':f'{text}', 'CallToAction':f'{button_txt}', 'ImageUrl':f'{img_url}'})

section class name : bf-acf-module image-and-content spacing-spaced image-right
{'Title': 'Local emergency plumbing services', 'content': 'Time is of the essence when it comes to plumbing emergencies. So, it really helps to have a local plumber nearby to assess how quickly the issue can be managed and resolved.  Our emergency plumbers are stationed throughout London, providing first-class services to customers across all boroughs, including areas in and around the M25. So, wherever you’re based in London, you can rely on us to have an emergency plumber near you. ', 'CallToAction': 'Book your local emergency plumber', 'ImageUrl': 'https://www.pimlicoplumbers.com/wp-content/uploads/2022/07/Emergency-plumber-image-content.jpg'}


In [157]:
section_to_search = sections[3]
class_name = " ".join(section_to_search['class'])
print(f"section class name : {class_name}")
if " ".join(section_to_search['class']) ==  'bf-acf-module customer-testimonials':
    title = section_to_search.find('h2')
    title = title.text
    img = section_to_search.find('img')
    img_url = img['src']
    testimonial_list = []
    test_div = section_to_search.find('div',{'class':'splide__list'})
    test_items = test_div.find_all('div', class_=lambda c: "splide__slide testimonial-slide" in c)
    for item in test_items:
        text = []
        h5 = item.find('div',{'class':'h5-style testimonial-item-title'})
        p = item.find('p',{'class':'testimonial-item-text'})
        author = item.find('div',{'class':'small-title testimonial-item-author'})
        text.append(h5.text.strip())
        text.append(p.text.strip())
        text.append(author.text.strip())
        test_txt = "\n".join(text)
        testimonial_list.append(test_txt)


section class name : bf-acf-module customer-testimonials
["Boiler replacement and radiator move very professional\nJames and Andrew spent 2 days replacing the boiler and replumbing radiators. Highly professional approach, keeping all their equipment tidy, insuring all replacement work was of the highest standard and fully compliant. It's not cheap, but you get real peace of mind with the work done.\nJamie", 'Quick to reply and fix the issues in an expert manner\nThe engineer was very experienced and sorted out the issues very soon. I am very happy with the services!\nJoanne Tipping', 'So impressed\nSo impressed - fantastic electrician Andrew fixed the kitchen under-cabinet lights that were flickering. He diagnosed and fixed the issue incredibly quickly, replacing the LED-driver unit with a brand new one and also fixed another electrical fault which he discovered.\n\nSuper friendly, professional and expert.\n\nHighly recommended 10/10\nDaniel H', 'The level of service and the response t